# Tuning

## Defining a Tuning Problem

A tuning problem in machine learning comes as choosing a set of
optimal `hyperparameters` for a `learning algorithm`.

By providing a set of `hyperparameters` and their range of values,
we have a `tuning problem` that can be solved by finding the optimal
values for those hyperparameters and obtain the best performance possible.


## What is a Hyperparameter?

In **BTB** a hyperparameter is a class that represents an object
which given a configuration, contains the methods that **BTB** needs
in order to find the optimal value for this parameter.

### Types of Hyperparameters

- `BooleanHyperParam`: boolean parameters i.e: `True` or `False`.
- `CategoricalHyperParam`: categorical parameters i.e: "foo", "bar".
- `FloatHyperParam`: `float` parameters i.e: `0.0 - 1.0`
- `IntHyperParam`: `int` parameters i.e: `0 - 1`

#### Creating a hyperparameter

In order to create a hyperparameter, we have to first import the
hyperparameter that we are about to use.

##### BooleanHyperParam

This hyperparameter is used for parameters that represent boolean values. This hyperparameter has the following
arguments:

- `default`: default value for the hyperparameter. Defaults to `False`.

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
from btb.tuning.hyperparams import BooleanHyperParam

bool_hp = BooleanHyperParam(default=True)

##### CategoricalHyperParam

This hyperparameter is used for hyperparameters that use categorical values. The following arguments
are accepted by this hyperparameter:
- `choices`: list of values that the hyperparameter can be.
- `default`: default value for the hyperparameter to take. Defaults to the first item in ``choices``.

In [3]:
from btb.tuning.hyperparams import CategoricalHyperParam

values = ['a', 'b', 'c']
categorical_hp = CategoricalHyperParam(choices=values, default='b')

##### FloatHyperParam

This hyperparameter is used for parameters that use `float` values.
It allows you to instantiate an open or a closed range of values
by providing the following arguments:

- `min` (float): minimum value that this hyperparameter can take, by default is ``None`` which will take the system's minimum float value possible.
- `max` (float): maximum value that this hyperparameter can take, by default is ``None`` which will take the system's maximum float value possible.
- `default` (float): number that represents the default value for the hyperparameter. Defaults to ``self.min``.
- `include_min` (bool): Either or not to include the minimum value, by default is ``True``.
- `include_max` (bool): Either or not to include the maximum value, by default is ``True``.

In [4]:
from btb.tuning.hyperparams import FloatHyperParam

float_hp = FloatHyperParam(min=0, max=1, default=0.5)

##### IntHyperParam

This hyperparameter is used for parameters that use `int` values.
It allows you to instantiate an open or a closed range of values
by providing the following arguments:

- `min` (int): minimum value that this hyperparameter can take, by default is ``None`` which will take the system's minimum int value possible.
- `max` (int): maximum value that this hyperparameter can take, by default is ``None`` which will take the system's maximum int value possible.
- `default` (int): number that represents the default value for the hyperparameter. Defaults to ``self.min``.
- `step` (int): Increase amount to take for each sample. Defaults to 1.
- `include_min` (bool): Either or not to include the minimum value, by default is ``True``.
- `include_max` (bool): Either or not to include the maximum value, by default is ``True``.

In [5]:
from btb.tuning.hyperparams import IntHyperParam

int_hp = IntHyperParam(min=1, max=10, default=5, include_min=False, include_max=True)

## What is Tunable?

`Tunable` is a class specifically designed to represent a tuning
problem. This class is a collection of `Hyperparameter` objects
and provides an api that **BTB** uses to solve a tuning problem.

### Creating a Tunable

#### Dictionary of hyperparams

In [6]:
from btb.tuning.tunable import Tunable
from btb.tuning.hyperparams import (
    BooleanHyperParam, CategoricalHyperParam, IntHyperParam, FloatHyperParam)

hyperparams = {
    'bhp': BooleanHyperParam(default=False),
    'chp': CategoricalHyperParam(choices=['foo', 'bar'], default='foo'),
    'fhp': FloatHyperParam(min=0, max=1, default=0.5),
    'ihp': IntHyperParam(min=1, max=10, default=2),
}

tunable = Tunable(hyperparams)

#### From a dictionary representing the hyperparameters

The class `Tunable` provides a method `from_dict` that accepts as an input a python dictionary
containing as `key` the given name for the hyperparameter and as value a dictionary containing
the following keys:

- `type` (str): ``bool`` for ``BoolHyperParam``, ``int`` for ``IntHyperParam``, ``float`` for ``FloatHyperParam``, ``str`` for ``CategoricalHyperParam``.
- `range` or `values` (list): range / values that this hyperparameter can take, in case of ``CategoricalHyperParam`` those will be used as the ``choices``, for ``NumericalHyperParams`` the ``min`` value will be used as the minimum value and the ``max`` value will be used as the ``maximum`` value.
- `default` (str, bool, int, float or None): The default value for the hyperparameter. 

The previously created `Tunable` can be created using the following dictionary:

In [7]:
hyperparams = {
    'bhp': {
        'type': 'bool',
        'default': False
    },
    'chp': {
        'type': 'str',
        'values': ['foo', 'bar'],
        'default': 'foo'
    },
    'fhp': {
        'type': 'float',
        'values': [0, 1],
        'default': 0.5
    },
    'ihp': {
        'type': 'int',
        'values': [1, 10],
        'default': 2
    }
}

tunable = Tunable.from_dict(hyperparams)

##  What is a Tuner?

Tuners are specifically designed to speed up the process of selecting the
optimal hyperparameter values for a specific machine learning problem.

``btb.tuning.tuners`` defines Tuners: classes with a fit/predict/propose interface for
suggesting sets of hyperparameters.

This is done by following a Bayesian Optimization approach and iteratively:

* letting the tuner propose new sets of hyper parameter
* fitting and scoring the model with the proposed hyper parameters
* passing the score obtained back to the tuner

At each iteration the tuner will use the information already obtained to propose
the set of hyper parameters that it considers that have the highest probability
to obtain the best results.

### Creating a Tuner

In order to create a `BTB` tuner, we will need a `Tunable` object that represents the
tuning problem. **Bear in mind** that you can import `Tunable` and the hyperparameters
from `btb.tuning`:

In [8]:
from btb.tuning import FloatHyperParam, IntHyperParam, Tunable

tuning_problem = Tunable({
    'fhp': FloatHyperParam(min=0, max=1),
    'ihp': IntHyperParam(min=1, max=10)
})

Now that we have our `tuning_problem` let's import a `GPTuner` from `btb.tuning.tuners` and create
a `Tuner` instance:

In [9]:
from btb.tuning.tuners import GPTuner

tuner = GPTuner(tuning_problem)

#### Tuner usage

The tuner is ment to be used with two methods:

##### Propose

This method will propose one or more new hyperparameter configuration(s)
by using the following aproach:
1. Create candidates.
2. Use acquisition function to acquire candidates.
3. Return the selected candidate to be evaluated

In [10]:
proposal = tuner.propose()
proposal

{'fhp': 0.8970714137989368, 'ihp': 7}

##### Record

This method will record the result of one trial. Then  it will
`re-fit` the meta-model in order to generate better proposals:
1. Append trial to internal results store.
2. Re-fit meta-model.

In [11]:
score = 0.5
tuner.record(proposal, score)

As you can see, those methods are ment to be used in a loop, that:

1. Propose.
2. Scores the proposal.
3. Records the proposal.

##### Tuning loop example

In order to create a `tuning loop` we will need a `scoring function` and
the `Tunable` for this `scoring function`. In this example, we will use the
mathematical function [Rosenbrock](https://en.wikipedia.org/wiki/Rosenbrock_function),
wich takes as input arguments: `x` and `y`.

Let's define this function in `python`:

In [12]:
def rosenbrock(x, y):
    return -1 * ((1 - x)**2 + 100 * (y - x**2)**2)


Now, let's create our `tunable` which will take values from `-50` to `50` for both `x` and `y`,
then, create a `Tuner` with that `tunable`:

In [13]:
from btb.tuning import GPTuner, IntHyperParam, Tunable

tunable = Tunable({
    'x': IntHyperParam(min=-50, max=50),
    'y': IntHyperParam(min=-50, max=50)
})

tuner = GPTuner(tunable)

Now we can create our tuning loop that will propose and record the obtained scores for 100 iterations: 

In [14]:
for _ in range(100):
    proposal = tuner.propose()
    score = rosenbrock(**proposal)
    tuner.record(proposal, score)

### Implemented tuners

**BTB** has the following three tuners available:
- [UniformTuner](https://github.com/HDI-Project/BTB/blob/master/btb/tuning/tuners/uniform.py): Uses a Tuner that samples proposals randomly using a uniform distribution.
- [GPTuner](https://github.com/HDI-Project/BTB/blob/master/btb/tuning/tuners/gaussian_process.py): Uses a Bayesian Tuner that optimizes proposals using a GaussianProcess metamodel.
- [GPEiTuner](https://github.com/HDI-Project/BTB/blob/master/btb/tuning/tuners/gaussian_process.py): Uses a Bayesian Tuner that optimizes proposals using a GaussianProcess metamodel and an Expected Improvement acquisition function.


### Tuners Leaderboard

Currently we have a [Benchmarking](https://github.com/HDI-Project/BTB/tree/master/benchmark)
process that evaluates the `tuners` performance against each other
this are the latest results that we obtained for the `BTB` tuners.


| tuner                   | with ties | without ties |
|-------------------------|-----------|--------------|
| `BTB.GPEiTuner`         |    **35** |            7 |
| `BTB.GPTuner`           |    33     |        **8** |
| `BTB.UniformTuner`      |    29     |            2 |